In [11]:
# !pip install transformers
# !pip install datasets
# !pip install trl
# !pip install torch
# !pip install torchdata
# !pip install peft
# !pip install matplotlib

In [12]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset


# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)


# model_name: str = "microsoft/phi-1_5"
# model = AutoModelForCausalLM.from_pretrained(
#     model_name, trust_remote_code=True, torch_dtype="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name, trust_remote_code=True, torch_dtype="auto"
# )

In [13]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text


def write_file(file_path, data):
    with open(file_path, "w") as f:
        for datum in data:
            f.write(datum)
            f.write("\n")

In [14]:
from transformers import TextDataset, DataCollatorForLanguageModeling


def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [15]:
import re

text_data_1 = read_txt("./lotr/the-fellowship-of-the-ring.txt")
text_data_2 = read_txt("./lotr/the-two-towers.txt")
text_data_3 = read_txt("./lotr/the-return-of-the-king.txt")

regex_pattern = r"\b[^.!?]+[.!?]+"
sentence_data_1 = re.findall(regex_pattern, text_data_1)
sentence_data_2 = re.findall(regex_pattern, text_data_2)
sentence_data_3 = re.findall(regex_pattern, text_data_3)
sentence_data = sentence_data_1 + sentence_data_2 + sentence_data_3

longest_sentence = 0
min_words_in_sentence = 0  # 10
max_string_sentence_len = 1024  # 512

clean_data = list()
for sentence in sentence_data:
    word_list = sentence.split()
    clean_sentence = " ".join(word_list)

    if len(word_list) > min_words_in_sentence:
        if len(clean_sentence) < max_string_sentence_len:
            clean_data.append(clean_sentence)

    if len(clean_sentence) > longest_sentence:
        longest_sentence = len(clean_sentence)

write_file("./lotr/all.txt", clean_data)

print("FIRST INDEX")
print(clean_data[0])
print("SECOND INDEX")
print(clean_data[1])
print("HOW MANY SENTENCES")
print(len(clean_data))
print("LONGEST SENTENCE")
print(longest_sentence)

FIRST INDEX
Three Rings for the Elven-kings under the sky, Seven for the Dwarf-lords in their halls of stone, Nine for Mortal Men doomed to die, One for the Dark Lord on his dark throne In the Land of Mordor where the Shadows lie.
SECOND INDEX
One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the darkness bind them In the Land of Mordor where the Shadows lie.
HOW MANY SENTENCES
35506
LONGEST SENTENCE
631


In [16]:
from transformers import (
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

In [17]:
# dataset = load_dataset(
#     "text",
#     data_files={
#         "train": [
#             "./lotr/all.txt",
#             # "./lotr/the-fellowship-of-the-ring.txt",
#             # "./lotr/the-return-of-the-king.txt",
#             # "./lotr/the-two-towers.txt",
#         ]
#     },
# )

# encoded_dataset = dataset.map(
#     lambda x: tokenizer.encode_plus(
#         x["text"],
#         add_special_tokens=True,
#         max_length=1024,
#         padding="max_length",
#         truncation=True,
#     ),
#     batched=True,
# )

In [19]:
model_name = "microsoft/phi-1_5"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True, torch_dtype="auto"
)

# tokenizer.save_pretrained(output_dir)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="./lotr/all.txt",
    block_size=1024,
)
print(dataset)

model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, torch_dtype="auto"
)

# model.save_pretrained(output_dir)

# train_loader = DataLoader(encoded_dataset["train"], batch_size=8, shuffle=True)
# eval_loader = DataLoader(encoded_dataset["test"], batch_size=8)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

optimizer = AdamW(model.parameters(), lr=5e-5)

training_args = TrainingArguments(
    output_dir="./output",
    # num_train_epochs=3,
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    # warmup_steps=500,
    # weight_decay=0.01,
    # logging_dir="./logs",
    # logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    # eval_dataset=dataset["test"],
)

trainer.train()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


KeyboardInterrupt: 

In [ ]:
# train_file_path = "./lotr/the-fellowship-of-the-ring.txt"
# model_name = "microsoft/phi-1_5"
# output_dir = "./output"
# overwrite_output_dir = True
# per_device_train_batch_size = 8
# num_train_epochs = 50.0
# save_steps = 50000

# train(
#     train_file_path=train_file_path,
#     model_name=model_name,
#     output_dir=output_dir,
#     overwrite_output_dir=overwrite_output_dir,
#     per_device_train_batch_size=per_device_train_batch_size,
#     num_train_epochs=num_train_epochs,
#     save_steps=save_steps,
# )

In [ ]:
# train_file_path = "./lotr/the-fellowship-of-the-ring.txt"
# model_name = "microsoft/phi-1_5"
# output_dir = "./output"
# overwrite_output_dir = True
# per_device_train_batch_size = 8
# num_train_epochs = 50.0
# save_steps = 50000

# train(
#     train_file_path=train_file_path,
#     model_name=model_name,
#     output_dir=output_dir,
#     overwrite_output_dir=overwrite_output_dir,
#     per_device_train_batch_size=per_device_train_batch_size,
#     num_train_epochs=num_train_epochs,
#     save_steps=save_steps,
# )

In [ ]:
# dataset = load_dataset(
#     "text",
#     data_files={
#         "train": [
#             "./lotr/the-fellowship-of-the-ring.txt",
#             "./lotr/the-return-of-the-king.txt",
#             "./lotr/the-two-towers.txt",
#         ]
#     },
# )


# def load_dataset(file_path, tokenizer, block_size=128):
#     dataset = TextDataset(
#         tokenizer=tokenizer,
#         file_path=file_path,
#         block_size=block_size,
#     )
#     return dataset


# print(f"Train dataset size: {len(dataset['train'])}")

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# inputs = tokenizer(
#     "write me a lord of the rings style short story",
#     return_tensors="pt",
#     return_attention_mask=False,
# )
# outputs = model.generate(**inputs, max_length=200)
# text = tokenizer.batch_decode(outputs)[0]
# print(text)

In [ ]:
# from peft import (
#     get_peft_config,
#     get_peft_model,
#     PromptTuningInit,
#     PromptTuningConfig,
#     TaskType,
#     PeftType,
#     PromptEncoderConfig,
#     PrefixTuningConfig,
#     LoraConfig,
#     PeftConfig,
# )

# lora_peft_config = LoraConfig()

# dynamic_padding = True


# def tokenize_func(examples):
#     return tokenizer(
#         examples["input"], truncation=True, max_length=1044
#     )  # max_length=512,  padding=True


# train_dataset_final = train_dataset.map(tokenize_func, batched=True)

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token
# model.resize_token_embeddings(len(tokenizer))
# model = prepare_model_for_int8_training(model)
# model = get_peft_model(model, lora_peft_config)
# training_args = model_training_args
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_dataset_final,
#     dataset_text_field="text",
#     max_seq_length=1044,
#     tokenizer=tokenizer,
#     args=model_training_args,
#     packing=True,
#     peft_config=lora_peft_config,
# )
# trainer.train()